<a href="https://colab.research.google.com/github/nina-adhikari/enjoyment-maximizing-maps/blob/main/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis

In this notebook, we take a look at some of the basic statistics of our data, looking to select the features we care about most. We are particularly looking for correlations among the features and output variables.

First, we do some administrative work:

In [ ]:
# packages that need to be installed
!pip install mapclassify #Choropleth map classification
!pip install ydata-profiling #Data profiling and exploratory data analysis

In [2]:
# packages that need to be loaded
import matplotlib.pyplot as plt #for the pandas profiling
import geopandas as gpd #extends the datatypes used by pandas to allow spatial operations on geometric types
import pandas as pd
import seaborn as sns #for pairplots
import numpy as np
from ydata_profiling import ProfileReport #automatic profiling of datasets
import pickle

In [3]:
# global variables to be defined
DIR = 'drive/MyDrive/walkability/'   # directory where all files are stored

In [4]:
#allow colab to access your Google Drive, an authentication window will pop-up
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
#open dictionary that contains the explanation of each column name
picklefile = DIR + 'cols_dict.pickle'
with open(picklefile, 'rb') as handle:
    cols_dict = pickle.load(handle)

In [ ]:
cols_dict

{'GEOID10': 'Census block group 12-digit FIPS code (2010)',
 'GEOID20': 'Census block group 12-digit FIPS code (2018)',
 'STATEFP': 'State FIPS code',
 'COUNTYFP': 'County FIPS cod',
 'TRACTCE': 'Census tract FIPS code in which CBG resides',
 'BLKGRPCE': 'Census block group FIPS code in which CBG resides',
 'CSA': 'Combined Statistical Area (CSA) Code',
 'CSA_Name': 'Name of CSA in which CBG resides',
 'CBSA': 'FIPS for Core-Based Statistical Area (CBSA) in which\nCBG resides',
 'CBSA_Name': 'Name of CBSA in which CBG resides',
 'CBSA_POP': 'Total population in CBSA',
 'CBSA_EMP': 'Total employment in CBSA',
 'CBSA_WRK': 'Total number of workers that live in CBSA',
 'Ac_Total': 'Total geometric area (acres) of the CBG',
 'Ac_Water': 'Total water area (acres)',
 'Ac_Land': 'Total land area (acres)',
 'Ac_Unpr': 'Total land area (acres) that is not protected from\ndevelopment (i.e., not a park, natural area or conservation\narea)',
 'TotPop': 'Population, 2018',
 'CountHU': 'Housing unit

Let us do a preliminary profiling to discover which variables are unique (those could index the data set), and discover which ones are categorical.

In [6]:
#unmodified dataset
df_full = gpd.read_file(DIR + 'maindata.gpkg')
df_full.sample(3)

,OBJECTID,GEOID10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,...,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST,Income,geometry
124,61468,110010098102,11,1,9810,2,548,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV",...,83.4,17.2,38.9,86.9,14.3,48.0,4.8,30.2,30704.0,"MULTIPOLYGON (((1622502.675 1918682.416, 16225..."
12,61356,110010091021,11,1,9102,1,548,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV",...,83.2,13.9,35.2,88.3,13.2,43.3,5.3,23.2,44217.0,"MULTIPOLYGON (((1622284.868 1929299.425, 16224..."
136,61480,110010092042,11,1,9204,2,548,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV",...,81.5,15.0,34.8,86.4,14.6,43.8,5.7,28.8,26215.0,"MULTIPOLYGON (((1621121.141 1928578.050, 16211..."


In [ ]:
#remove geometric information before profiling
geometry_cols = ['Shape_Length', 'Shape_Area', 'geometry']

#profiling
profile_full = ProfileReport(df_full.drop(columns=geometry_cols), title="Pandas Preliminary Profiling Report", minimal=True) #minimal will give hist and descriptive statistics
profile_full


*   There are three text variables: OBJECTID, CSA_Name, CBSA_Name.
*   Because we are working with DC only, there are several constant cols:  STATEFP, COUNTYFP, CSA, CSA_Name, CBSA, CBSA_Name, CBSA_POP, CBSA_EMP, CBSA_WRK
*   Since GEOID10 has unique values, it can be used to index the dataset. It corresponds to the 12-digit FIPS Census block group code (2010). OBJECTID can also be used.

Let us remove some other redundant cols...




In [8]:
# Columns we ignore when doing numerical operations (such as Name etc). Also, since we are only interested in DC, we will also drop CBSA/county/state data
non_numeric_columns = ['GEOID10', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'CSA', 'CSA_Name', 'CBSA', 'CBSA_Name', 'CBSA_POP', 'CBSA_EMP', 'CBSA_WRK', 'Shape_Length', 'Shape_Area', 'geometry']

# Keeping track of the health outcome columns
health_columns = ['ACCESS2', 'ARTHRITIS', 'BINGE', 'BPHIGH', 'BPMED', 'CANCER', 'CASTHMA', 'CHD', 'CHECKUP', 'CHOLSCREEN', 'COLON_SCREEN', 'COPD', 'COREM', 'COREW', 'CSMOKING', 'DENTAL', 'DIABETES', 'HIGHCHOL', 'KIDNEY', 'LPA', 'MAMMOUSE', 'MHLTH', 'OBESITY', 'PAPTEST', 'PHLTH', 'SLEEP', 'STROKE', 'TEETHLOST']

In [9]:
df_full['OBJECTID'] = df_full['OBJECTID'].astype(int)

#df is data without non-numeric cols
df = df_full.drop(columns=non_numeric_columns)
df_nn = df_full[non_numeric_columns]

df.set_index('OBJECTID', inplace=True)
df = df.apply(pd.to_numeric, errors='ignore')

# Some of the entries are set to -99999 (see the PDF, page 23 footnote 65), so we will replace them (and in fact all negative entries) with NaN:
df[df < 0] = np.nan
df.sample(5)

,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,P_WrkAge,AutoOwn0,Pct_AO0,...,LPA,MAMMOUSE,MHLTH,OBESITY,PAPTEST,PHLTH,SLEEP,STROKE,TEETHLOST,Income
OBJECTID,,,,,,,,,,,,,,,,,,,,,
61360,54.278233,0.0,54.278233,54.231276,1423,876,782,0.751,195,0.249361,...,27.3,82.5,12.0,29.5,87.5,10.5,39.7,3.9,15.8,70156.0
61652,18.149248,0.0,18.149248,17.112824,852,699,648,0.830,216,0.333333,...,17.6,82.3,7.5,21.7,89.6,7.1,31.2,2.7,5.7,75529.0
61524,122.469679,0.0,122.469679,122.416915,2061,895,875,0.525,336,0.384000,...,14.8,80.5,7.8,17.0,86.8,6.0,27.9,1.8,4.8,164180.0
61635,18.566600,0.0,18.566600,18.566600,1204,720,648,0.854,369,0.569444,...,14.8,82.2,8.6,18.6,89.0,5.3,30.5,1.3,5.2,97557.0
61672,60.567175,0.0,60.567175,57.273739,1294,435,371,0.798,26,0.070081,...,16.1,81.4,9.3,18.2,87.9,5.8,30.5,1.4,7.0,124583.0


We first use the standard (Pearson) correlation to discover some redundant features:

In [10]:
from collections import Counter

CDC_THRESHOLD = 0.95
EPA_THRESHOLD = 0.8

def get_cols_to_drop(dataframe, threshold):
  corr = dataframe.corr().abs()                                     # correlation matrix
  corr.values[np.triu_indices_from(corr, 0)] = np.nan               # since corr is symmetric, work only with upper triangular part
  stacked_corr = corr.stack()
  corr_dict = dict(stacked_corr.loc[stacked_corr > threshold])
  counter = Counter(sum(corr_dict.keys(), ()))
  if len(counter) == 0:
    return []
  most_common_col = counter.most_common()[0][0]
  #print(most_common_col)
  columns_to_drop = set([])
  for key in corr_dict.keys():
    #print(list(key))
    if most_common_col in key:
      columns_to_drop.add(list(key)[1-key.index(most_common_col)])
  return list(columns_to_drop)

def iterated_cols_to_drop(dataframe, threshold):
  all_cols = set()
  while(True):
    cols = get_cols_to_drop(dataframe, threshold)
    all_cols = all_cols.union(set(cols))
    #print(cols)
    if len(cols) == 0:
      break
    dataframe.drop(columns=cols, inplace=True)
  return all_cols

df_health = df[health_columns].copy()

health_cols_to_drop = iterated_cols_to_drop(df_health, CDC_THRESHOLD)

epa_cols_to_drop = iterated_cols_to_drop(df.drop(columns=health_columns), EPA_THRESHOLD)

health_cols_to_drop, epa_cols_to_drop

({'ACCESS2',
  'ARTHRITIS',
  'BINGE',
  'CASTHMA',
  'CHECKUP',
  'COLON_SCREEN',
  'COPD',
  'COREW',
  'CSMOKING',
  'DIABETES',
  'KIDNEY',
  'LPA',
  'OBESITY',
  'SLEEP',
  'STROKE',
  'TEETHLOST'},
 {'Ac_Land',
  'Ac_Water',
  'AutoOwn0',
  'AutoOwn1',
  'D3B',
  'D5BR',
  'E5_Ent',
  'E5_Ind',
  'E5_Off',
  'E5_Ret',
  'E5_Svc',
  'E8_Ent',
  'E8_Svc',
  'E8_off',
  'E_HiWageWk',
  'E_LowWageWk',
  'HH',
  'R_LowWageWk',
  'TotEmp',
  'Workers'})

Based on the above analysis we will drop several non-health columns. To do this, we select 10 outcomes to focus on preliminaryly, seven from outside the correlation list above and four from the list.

In [11]:
health_columns_to_keep = ['BPHIGH', 'CANCER', 'CHD', 'DENTAL', 'DIABETES', 'HIGHCHOL', 'MHLTH', 'LPA', 'PHLTH', 'STROKE', 'OBESITY']

df.drop(columns=epa_cols_to_drop, inplace=True, errors='ignore')
df.drop(columns=[x for x in health_columns if x not in health_columns_to_keep], inplace=True, errors='ignore')
df.columns

Index(['Ac_Total', 'Ac_Unpr', 'TotPop', 'CountHU', 'P_WrkAge', 'Pct_AO0',
       'Pct_AO1', 'AutoOwn2p', 'Pct_AO2p', 'R_MedWageWk', 'R_HiWageWk',
       'R_PCTLOWWAGE', 'E8_Ret', 'E8_Ind', 'E8_Ed', 'E8_Hlth', 'E8_Pub',
       'E_MedWageWk', 'E_PctLowWage', 'D3A', 'D3AAO', 'D3AMM', 'D3APO',
       'D3BAO', 'D3BMM3', 'D3BMM4', 'D3BPO3', 'D3BPO4', 'D4A', 'D4B025',
       'D4B050', 'D4C', 'D5AR', 'D5AE', 'D5BE', 'NatWalkInd', 'BPHIGH',
       'CANCER', 'CHD', 'DENTAL', 'DIABETES', 'HIGHCHOL', 'LPA', 'MHLTH',
       'OBESITY', 'PHLTH', 'STROKE', 'Income'],
      dtype='object')

Let's do a **not minimal** profiling in the columns that are to be our output: health_columns_to_keep in order to discover further correlations

In [ ]:
#profiling only health columns
profile_health = ProfileReport(df_full[health_columns_to_keep], title="Pandas Preliminary Profiling Report")
profile_health

# Health outcomes analysis

From the interactions section above, it is clear that many variables are highly correlated. Several of these correlations are known from health research. In fact, for many of them there is a causality relation. Thus, we aim to get rid of the ones that are more a consequence of another one. Also, we focus on the ones that are most costly for America. We do this as follows:

*   The most expensive conditions in terms of direct health care costs are **diabetes**, Alzheimer’s, and osteoarthritis.
*   The most common chronic health conditions in the U.S. are hypertension (**high blood pressure**), dyslipidemia, and osteoarthritis.
*   **Obesity** is by far the greatest risk factor contributing to the burden of chronic diseases in the U.S.
*   The leading causes of death in the US among the tracked variables are **heart disease**, **cancer**, **stroke**, and **diabetes**
*   The following are highly correlated in our dataset and are known to have these causality relationships: BPHIGH -> CHD, DIABETES -> BPHIGH, DIABETES -> CHD, HIGHCHOL-> CHD, BPHIGH -> STROKE, CHD -> STROKE, DIABETES -> STROKE
*   Also, PHLTH is too general, so we get rid of it
*   Since DENTAL seems to be a consequence of a generally good lifestyle, we also chose to omit it.

Based on this analysis, we keep the following as health outcomes: DIABETES, BPHIGH, OBESITY, CANCER, LPA, MHLTH


In [12]:
#add all health cols to drop, but the ones above
cols_to_drop = []
cols_to_drop += ['CHD', 'DENTAL', 'HIGHCHOL', 'PHLTH', 'STROKE']

We will now use Seaborn pair plots to find correlations between columns and remove the redundant (or mostly redundant) ones.

In [ ]:
# Setting the stlying of the Seaborn figure
sns.set_style('darkgrid')

Pair plots of D4 and D5:

In [ ]:
cols_to_plot = ['D4A', 'D4B025', 'D4B050', 'D4C', 'D5AR', 'D5AE', 'D5BE']
sns.pairplot(df[cols_to_plot], diag_kind='kde')

There is a correlation between D5AR and D5AE

In [13]:
cols_to_drop += ['D5AE']

Pair plots of auto ownership:

In [ ]:
cols_to_plot = ['Pct_AO0', 'Pct_AO1', 'AutoOwn2p', 'Pct_AO2p']
sns.pairplot(df[cols_to_plot], diag_kind='kde')

As expected, there are correlations between # and % households having fixed # of cars



In [14]:
cols_to_drop += ['AutoOwn2p']

Population and employment:

In [ ]:
cols_to_plot = ['TotPop', 'CountHU', 'P_WrkAge']
sns.pairplot(df[cols_to_plot], diag_kind='kde')

Correlation between CountHU and TotPop

In [15]:
cols_to_drop += ['CountHU']

Distribution of workers based on income:

In [ ]:
cols_to_plot = ['R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE']
sns.pairplot(df[cols_to_plot], diag_kind='kde')

In [16]:
cols_to_drop += ['R_HiWageWk']

In [17]:
cols_to_drop

['CHD',
 'DENTAL',
 'HIGHCHOL',
 'PHLTH',
 'STROKE',
 'D5AE',
 'AutoOwn2p',
 'CountHU',
 'R_HiWageWk']

In [18]:
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
df

,Ac_Total,Ac_Unpr,TotPop,P_WrkAge,Pct_AO0,Pct_AO1,Pct_AO2p,R_MedWageWk,R_PCTLOWWAGE,E8_Ret,...,D5AR,D5BE,NatWalkInd,BPHIGH,CANCER,DIABETES,LPA,MHLTH,OBESITY,Income
OBJECTID,,,,,,,,,,,,,,,,,,,,,
61344,16.948095,16.948095,553,0.821,0.520286,0.427208,0.052506,34,0.102102,121,...,358602,295441.0,17.166667,15.1,4.0,3.3,12.6,7.5,16.5,102455.0
61345,25.638849,24.554800,1210,0.755,0.342179,0.532123,0.125698,73,0.099828,0,...,373993,296090.0,17.833333,21.8,6.5,5.2,14.3,7.0,17.3,143586.0
61346,23.832548,23.832548,1188,0.879,0.433409,0.436795,0.129797,106,0.101863,94,...,402080,324820.0,17.500000,14.1,3.6,3.1,12.7,7.8,16.1,105978.0
61347,16.884343,16.884343,1150,0.792,0.657778,0.321111,0.021111,85,0.122449,92,...,397964,363001.0,15.666667,14.7,3.7,3.2,13.6,8.0,16.9,90402.0
61348,9.810858,9.810858,1739,0.807,0.659735,0.340265,0.000000,162,0.151786,0,...,395250,380231.0,13.166667,19.3,3.3,5.3,19.7,10.3,22.2,87969.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61789,18.081790,18.081790,1045,0.891,0.682598,0.221814,0.095588,126,0.157980,0,...,332374,169268.0,12.333333,15.7,4.0,3.7,16.4,10.0,18.2,71671.0
61790,19.805559,18.978260,1080,0.711,0.221906,0.778094,0.000000,67,0.109589,0,...,327595,172329.0,12.000000,24.6,7.4,6.2,17.1,8.1,18.3,89889.0
61791,125.755120,108.547510,573,0.661,0.137195,0.536585,0.326220,44,0.114144,0,...,292105,312928.0,13.166667,18.6,5.4,4.4,14.6,8.0,17.0,113300.0


Next we drop the columns with too many zeroes

In [19]:
ZERO_THRESHOLD = 0.4

zeroes = df.isin([0]).sum(axis=0) / len(df)
zeroes
columns_with_zeroes = [x for x in zeroes.axes[0] if zeroes[x] > ZERO_THRESHOLD]
df.drop(columns=columns_with_zeroes, inplace=True, errors='ignore')
df

,Ac_Total,Ac_Unpr,TotPop,P_WrkAge,Pct_AO0,Pct_AO1,Pct_AO2p,R_MedWageWk,R_PCTLOWWAGE,E8_Ret,...,D5AR,D5BE,NatWalkInd,BPHIGH,CANCER,DIABETES,LPA,MHLTH,OBESITY,Income
OBJECTID,,,,,,,,,,,,,,,,,,,,,
61344,16.948095,16.948095,553,0.821,0.520286,0.427208,0.052506,34,0.102102,121,...,358602,295441.0,17.166667,15.1,4.0,3.3,12.6,7.5,16.5,102455.0
61345,25.638849,24.554800,1210,0.755,0.342179,0.532123,0.125698,73,0.099828,0,...,373993,296090.0,17.833333,21.8,6.5,5.2,14.3,7.0,17.3,143586.0
61346,23.832548,23.832548,1188,0.879,0.433409,0.436795,0.129797,106,0.101863,94,...,402080,324820.0,17.500000,14.1,3.6,3.1,12.7,7.8,16.1,105978.0
61347,16.884343,16.884343,1150,0.792,0.657778,0.321111,0.021111,85,0.122449,92,...,397964,363001.0,15.666667,14.7,3.7,3.2,13.6,8.0,16.9,90402.0
61348,9.810858,9.810858,1739,0.807,0.659735,0.340265,0.000000,162,0.151786,0,...,395250,380231.0,13.166667,19.3,3.3,5.3,19.7,10.3,22.2,87969.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61789,18.081790,18.081790,1045,0.891,0.682598,0.221814,0.095588,126,0.157980,0,...,332374,169268.0,12.333333,15.7,4.0,3.7,16.4,10.0,18.2,71671.0
61790,19.805559,18.978260,1080,0.711,0.221906,0.778094,0.000000,67,0.109589,0,...,327595,172329.0,12.000000,24.6,7.4,6.2,17.1,8.1,18.3,89889.0
61791,125.755120,108.547510,573,0.661,0.137195,0.536585,0.326220,44,0.114144,0,...,292105,312928.0,13.166667,18.6,5.4,4.4,14.6,8.0,17.0,113300.0


Let's do a minimal profiling, to explore some outliers to get rid of.

In [ ]:
profile = ProfileReport(df, title="Pandas Preliminary Profiling Report", variables={'descriptions': cols_dict}, minimal=True) #minimal will give hist and descriptive statistics
profile

In [ ]:
#what are the outliers?
df[(df['Ac_Total'] > 475)].index.values.tolist()

[61372, 61405, 61476, 61613, 61619, 61627, 61670, 61682, 61695, 61721]

In [ ]:
df[(df['Ac_Unpr'] > 250)].index.values.tolist()

[61357, 61372, 61405, 61406, 61408, 61489, 61619, 61627, 61711, 61721]

It seems like the ones that are outliers for several columns are the same. Let's get rid of those rows systematically, like this: For each column, get the list of IDs that are the $t$ top values. Then we do a histogram of those ID, and decide which rows to get rid of...after that we run the profiling again.

In [20]:
#decide on a good cutoff number
t = 9 #t largest of col
common_outliers = []
for column in df:
    common_outliers += df.nlargest(t, column).index.values.tolist()
    common_outliers += df.nsmallest(t, column).index.values.tolist()
    #print(common_outliers)

common_outliers_count = Counter(common_outliers)
print(common_outliers_count)

#plt.hist(common_outliers)

Counter({61486: 12, 61619: 10, 61348: 10, 61426: 8, 61372: 7, 61695: 7, 61627: 7, 61347: 7, 61476: 6, 61582: 6, 61489: 6, 61593: 6, 61592: 6, 61477: 6, 61354: 6, 61417: 6, 61614: 6, 61603: 6, 61373: 6, 61675: 6, 61355: 6, 61460: 6, 61660: 5, 61487: 5, 61550: 5, 61594: 5, 61737: 5, 61351: 5, 61419: 5, 61361: 5, 61371: 5, 61362: 5, 61461: 5, 61389: 5, 61698: 5, 61721: 4, 61670: 4, 61653: 4, 61581: 4, 61711: 4, 61440: 4, 61344: 4, 61607: 4, 61567: 4, 61391: 4, 61424: 4, 61678: 4, 61676: 4, 61349: 4, 61732: 4, 61352: 4, 61384: 4, 61377: 4, 61551: 4, 61765: 4, 61405: 3, 61624: 3, 61733: 3, 61473: 3, 61793: 3, 61406: 3, 61437: 3, 61521: 3, 61515: 3, 61480: 3, 61484: 3, 61400: 3, 61692: 3, 61490: 3, 61488: 3, 61393: 3, 61491: 3, 61438: 3, 61367: 3, 61376: 3, 61379: 3, 61449: 3, 61738: 3, 61483: 3, 61686: 3, 61422: 3, 61459: 3, 61595: 3, 61346: 3, 61585: 3, 61681: 3, 61682: 2, 61408: 2, 61357: 2, 61436: 2, 61655: 2, 61752: 2, 61428: 2, 61734: 2, 61634: 2, 61388: 2, 61609: 2, 61410: 2, 61432: 2

In [21]:
df_geom = df_full[['OBJECTID', 'GEOID10', 'geometry']]
df_geom.set_index('OBJECTID', inplace=True)
df_geom

,GEOID10,geometry
OBJECTID,,
61344,110010040023,"MULTIPOLYGON (((1617361.678 1927352.465, 16173..."
61345,110010041001,"MULTIPOLYGON (((1617028.696 1927124.163, 16170..."
61346,110010042022,"MULTIPOLYGON (((1617427.574 1927158.696, 16174..."
61347,110010053014,"MULTIPOLYGON (((1617857.365 1926906.416, 16178..."
61348,110010050021,"MULTIPOLYGON (((1618967.176 1926639.840, 16189..."
...,...,...
61789,110010007022,"MULTIPOLYGON (((1614569.609 1927830.272, 16146..."
61790,110010007012,"MULTIPOLYGON (((1614183.231 1928298.295, 16141..."
61791,110010013023,"MULTIPOLYGON (((1615366.380 1930018.785, 16153..."


In [ ]:
df = df.join(other=df_geom, on='OBJECTID', validate='1:1')
df = gpd.GeoDataFrame(df)
df

Save dataframe to file:

In [24]:
df = df.apply(pd.to_numeric, errors='ignore')
df.to_file("drive/MyDrive/walkability/post-eda.gpkg", driver="GPKG")

Check that the dataframe is the same as the data in the file:

In [25]:
new_df = gpd.read_file("drive/MyDrive/walkability/post-eda.gpkg")
new_df.set_index('OBJECTID', inplace=True)
new_df = new_df.apply(pd.to_numeric, errors='ignore')
new_df.sort_index(axis=1).equals(df.sort_index(axis=1))

True